#  IMT574 Problem Set 5: Naïve Bayes

### Group :Yash Raichura, Ganapathy SL, Roshni Roy


In [269]:
#Importing the libraries

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from scipy import stats

### Examining the data

In [270]:
#Importing the dataset

data=pd.read_csv("rotten-tomatoes.csv")

In [271]:
#Top 5 rows in the dataset
print(data.head(5))

,critic,fresh,imdb,link,publication,quote,review_date,rtid,title
0,Derek Adams,fresh,114709,http://www.timeout.com/film/reviews/87745/toy-...,Time Out,"So ingenious in concept, design and execution ...",2009-10-04 00:00:00,9559,Toy Story
1,Richard Corliss,fresh,114709,"http://www.time.com/time/magazine/article/0,91...",TIME Magazine,The year's most inventive comedy.,2008-08-31 00:00:00,9559,Toy Story
2,David Ansen,fresh,114709,http://www.newsweek.com/id/104199,Newsweek,A winning animated feature that has something ...,2008-08-18 00:00:00,9559,Toy Story
3,Leonard Klady,fresh,114709,http://www.variety.com/review/VE1117941294.htm...,Variety,The film sports a provocative and appealing st...,2008-06-09 00:00:00,9559,Toy Story
4,Jonathan Rosenbaum,fresh,114709,http://onfilm.chicagoreader.com/movies/capsule...,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10 00:00:00,9559,Toy Story
5,Michael Booth,fresh,114709,http://www.denverpost.com/movies/ci_5786068,Denver Post,"As Lion King did before it, Toy Story revived ...",2007-05-03 00:00:00,9559,Toy Story
6,Geoff Andrew,fresh,114709,http://www.timeout.com/film/reviews/79673/toy_...,Time Out,The film will probably be more fully appreciat...,2006-06-24 00:00:00,9559,Toy Story
7,Janet Maslin,fresh,114709,http://movies.nytimes.com/movie/review?res=990...,New York Times,Children will enjoy a new take on the irresist...,2003-05-20 00:00:00,9559,Toy Story
8,Kenneth Turan,fresh,114709,http://www.calendarlive.com/movies/reviews/cl-...,Los Angeles Times,Although its computer-generated imagery is imp...,2001-02-13 00:00:00,9559,Toy Story
9,Roger Ebert,fresh,114709,http://www.rogerebert.com/reviews/toy-story-1995,Chicago Sun-Times,The result is a visionary roller-coaster ride ...,2000-01-01 00:00:00,9559,Toy Story


In [272]:
#Bottom 5 rows in the dataset
print(data.tail(5))

,critic,fresh,imdb,link,publication,quote,review_date,rtid,title
13432,Rita Kempley,rotten,104389,http://www.washingtonpost.com/wp-srv/style/lon...,Washington Post,This anti-feminist parable is both a labor and...,2000-01-01 00:00:00,14013,The Hand That Rocks the Cradle
13433,Richard Corliss,fresh,88683,"http://www.time.com/time/magazine/article/0,91...",TIME Magazine,"All three stars do smart, honorable work.",2013-05-08 00:00:00,11917,Agnes of God
13434,Kevin Thomas,rotten,88683,http://articles.latimes.com/1985-09-13/enterta...,Los Angeles Times,While Agnes of God has been considerably opene...,2013-05-08 00:00:00,11917,Agnes of God
13435,Dave Kehr,rotten,88683,http://www.chicagoreader.com/chicago/agnes-of-...,Chicago Reader,"Despite all the anguished huffing and puffing,...",2013-05-08 00:00:00,11917,Agnes of God
13436,Jay Boyar,fresh,88683,http://articles.orlandosentinel.com/1985-10-30...,Orlando Sentinel,It is Meg Tilly who makes the movie live. Her ...,2013-05-08 00:00:00,11917,Agnes of God
13437,Gene Siskel,rotten,88683,http://articles.chicagotribune.com/1985-09-13/...,Chicago Tribune,Agnes of God plays with some challenging ideas...,2013-05-08 00:00:00,11917,Agnes of God
13438,Variety Staff,rotten,88683,http://www.variety.com/review/VE1117796703.htm...,Variety,"Fonda's relentless interrogating, mannered cha...",2008-10-18 00:00:00,11917,Agnes of God
13439,NaN,fresh,88683,http://www.timeout.com/film/reviews/77605/agne...,Time Out,Splendidly shot by Sven Nykvist and with excel...,2006-06-24 00:00:00,11917,Agnes of God
13440,Janet Maslin,rotten,88683,http://movies.nytimes.com/movie/review?res=950...,New York Times,"Miss Tilly makes a radiant Agnes, and Miss Ban...",2003-05-20 00:00:00,11917,Agnes of God
13441,Roger Ebert,rotten,88683,http://www.rogerebert.com/reviews/agnes-of-god...,Chicago Sun-Times,Although the movie deals in the basic material...,2000-01-01 00:00:00,11917,Agnes of God


In [273]:
#Columns in the dataset
print(data.shape, "\n")

#Types of columns
print(data.dtypes)

(13442, 9) 

critic         object
fresh          object
imdb            int64
link           object
publication    object
quote          object
review_date    object
rtid            int64
title          object
dtype: object


#### Printing the variable names

In [274]:
#Variables in the dataset
print(data.columns)

Index(['critic', 'fresh', 'imdb', 'link', 'publication', 'quote',
       'review_date', 'rtid', 'title'],
      dtype='object')


### Summary Statistics

#### a) Number of missings for fresh and quote

In [275]:
#Missings in fresh
print("Missing values in fresh:", data.fresh.isna().sum())

#Missings in quote
print("Missing values in quote:", data.quote.isna().sum())


Missing values in fresh: 0
Missing values in quote: 0


#### b) Different values for fresh/rotten evaluations 

In [276]:
#Different values of fresh
data.fresh.unique()

array(['fresh', 'rotten', 'none'], dtype=object)

#### c) Counts and percentages of different fresh values

In [277]:
#Different fresh and rotten values
print("Counts and percentages of different fresh values")
percentages=pd.DataFrame(data.groupby('fresh')['fresh'].count())
percentages.columns=['count']
percentages['percentage']=percentages.apply(lambda count: 100*count/float(count.sum()))
percentages

Counts and percentages of different fresh values


,count,percentage
fresh,,
fresh,8389,62.408868
none,23,0.171105
rotten,5030,37.420027


#### d) number of zero-length or only whitespace quote-s

In [278]:
#Number of zero-length value

print('Number of zero-length values', (data.quote.apply(lambda x : len(x)) == 0).sum())


#Number of white space values
quotes=pd.DataFrame(data['quote'])
quotes['Length']=quotes['quote'].str.len()


print("Length of quotes")
print(quotes.head(5))

print("Num of white space quotes")
print(quotes['quote'].str.isspace().sum())


Number of zero-length values 0
Length of quotes
                                               quote  Length
0  So ingenious in concept, design and execution ...     137
1                  The year's most inventive comedy.      33
2  A winning animated feature that has something ...      79
3  The film sports a provocative and appealing st...     107
4  An entertaining computer-generated, hyperreali...     110
Num of white space quotes
0


#### e) Minimum-maximum-average length of quotes (either in words, or in characters)

In [279]:
#In terms of characters
#Maximum length of quotes(in characters)
print("Maximum number of characters : ",quotes['Length'].max())

#Minimum length of quotes(in characters)
print("Minimum number of characters : ", quotes['Length'].min())

#Average number of characters
avg=round(sum(quotes['Length'])/len(quotes['Length']))
print("Average number of characters : ", avg)

Maximum number of characters :  256
Minimum number of characters :  4
Average number of characters :  121


In [280]:
#In terms of words
#Maximum length of quotes(in words)
split_quotes_max = len(max(quotes["quote"].str.split(" ").to_list(), key=len))
print("Maximum length in terms of words:", split_quotes_max)


#Minimum length of quotes(in words)
split_quotes_min = len(min(quotes["quote"].str.split(" ").to_list(), key=len))
print("Minimum length in terms of words:", split_quotes_min)



#Average number of words
print("Average length in terms of words:", 
      round(sum(map(len,quotes["quote"].str.split(" ") ))/len(quotes["quote"].str.split(" "))))

Maximum length in terms of words: 49
Minimum length in terms of words: 1
Average length in terms of words: 20


#### f)  How many reviews are in data multiple times

In [281]:
mult_reviews = (data.groupby('quote', as_index = False).size().
                sort_values(ascending = False).reset_index(name = 'Count of occurences'))

In [282]:
mult_reviews.columns

Index(['quote', 'count'], dtype='object')

#### Count of occurences is indicating the number of times a quote occurs. Number of quotes indicates how many quotes are being repeated. 1 quote occurs 4 times, 45 quotes appear 3 times, 513 quotes occur twice and 12277 quotes occur once

In [283]:
mult_reviews.groupby('Count of occurences', as_index = False).size().sort_values().reset_index(name = 'Number of quotes')

,count,counts
0,4,1
1,3,45
2,2,513
3,1,12277


In [284]:
data = data[data['fresh'] != 'none']
data = data.drop_duplicates()
data = data.reset_index()
data = data.drop(columns = 'index')
data.head()

,critic,fresh,imdb,link,publication,quote,review_date,rtid,title
0,Derek Adams,fresh,114709,http://www.timeout.com/film/reviews/87745/toy-...,Time Out,"So ingenious in concept, design and execution ...",2009-10-04 00:00:00,9559,Toy Story
1,Richard Corliss,fresh,114709,"http://www.time.com/time/magazine/article/0,91...",TIME Magazine,The year's most inventive comedy.,2008-08-31 00:00:00,9559,Toy Story
2,David Ansen,fresh,114709,http://www.newsweek.com/id/104199,Newsweek,A winning animated feature that has something ...,2008-08-18 00:00:00,9559,Toy Story
3,Leonard Klady,fresh,114709,http://www.variety.com/review/VE1117941294.htm...,Variety,The film sports a provocative and appealing st...,2008-06-09 00:00:00,9559,Toy Story
4,Jonathan Rosenbaum,fresh,114709,http://onfilm.chicagoreader.com/movies/capsule...,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10 00:00:00,9559,Toy Story


### Data Cleaning

In [285]:
#Dropping NA values if any in the fresh and quote column
def CheckInconsistency (dataset,variable1, variable2):
    # check if variable 1 and variable 2 are not missing.
    #print(variable1)
    if ((dataset[variable1].isna().sum()>0 ) | (dataset[variable1].isnull().sum()>0) |(dataset[variable2].isna().sum()>0 )|(dataset[variable2].isnull().sum()>0)):
        print('There are inconsistencies. hence dropping the null and NA values.')
        dataset=dataset[variable1].dropna()
        dataset=dataset[variable2].dropna()
    else:
        print('There are no inconsistencies in '+ variable1 + ' and ' + variable2 + ' columns')
    
    #check if quote is not an empty string
    if ((dataset[variable1].empty == True) | ((dataset[variable1]== '').sum()>0) | (dataset[variable1].astype(str).str.isspace().sum())):
        print('There are empty syring values ')
        
        a=dataset[dataset[variable1]==''].index
        dataset.drop(dataset.index(a))
    else:
        print('There are no empty values ')
    
    #Drop the duplicate values
    
    # keep first duplicate row
    dataset = dataset.drop_duplicates()

In [286]:
#Checking for the stopwords and removing numbers, punctuations , whitespaces
import re
from nltk.corpus import stopwords

def quote_to_words(raw_quote):
    #removing raw letters,numbers,punctuations
    letters = re.sub("[^a-zA-Z]"," ",raw_quote)
    #creating an array , resolving whitespaces
    words = letters.lower().split()
    #create an array of stopwords so that we don't have to access corpus to search for a stopword
    stop = set(stopwords.words("english"))
    #removing stopwords from the raw_review
    meaningful_words = [w for w in words if w not in stop]
    #return a string with only the words that are important
    return(" ".join(meaningful_words))

In [287]:
CheckInconsistency(data, 'fresh' , 'quote')

There are no inconsistencies in fresh and quote columns
There are no empty values 


In [288]:
clean_quotes = []

for i in range(data.quote.size):
    clean_quotes.append(quote_to_words(data.quote[i]))


quotes_series = pd.Series(clean_quotes)

#Creating work data
work_data = pd.DataFrame(quotes_series)

#Combining the fresh and quotes columns
work_data.rename(columns = {0 : 'quote'}, inplace = True)

#Appending the fresh column
work_data['fresh'] = data.fresh

In [ ]:
work_data.head(5)

In [290]:
#Number of quotes
work_data.shape

(12823, 2)

In [291]:
#Implementing vectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary = True)
X = vectorizer.fit_transform(clean_quotes).toarray()
X.shape


y = data[['fresh']] # Target
words = vectorizer.get_feature_names()

#Number of unique words
len(set(words))


20492

In [292]:
#Splitting the data into train and test models

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)



In [320]:
#Calculating probabilities of fresh and rotten with log probabilities
prob_fresh = y_train.fresh[y_train.fresh == 'fresh'].count()/len(y_train)
prob_rotten = 1 - prob_fresh

log_prob_fresh = np.log(prob_fresh)
log_prob_rotten = np.log(prob_rotten)
print(log_prob_fresh)
print(log_prob_rotten)

-0.48165241862569685
-0.961711165656882


In [293]:

y_train.reset_index(inplace= True)
y_train.drop(columns = 'index', inplace = True)


#Calculating the probabilities of words given fresh

index_fresh = y_train[y_train['fresh'] == 'fresh'].index
X_train_fresh = X_train[index_fresh].sum(axis = 0)
prob_w_f = X_train_fresh/len(y_train[y_train['fresh'] == 'fresh'])


#Calculating conditional probabilites for Word|Rotten 
index_rotten = y_train[y_train['fresh'] == 'rotten'].index
X_train_rotten = X_train[index_rotten].sum(axis = 0)
prob_w_r = X_train_rotten/len(y_train[y_train['fresh'] == 'rotten'])

C:\Users\roshn\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [302]:
#Inserting the calculated prob P(W|R) and P(W|F) into rotten and fresh df as all set of words as columns
fresh = pd.DataFrame(pd.Series(words)).reset_index()
fresh.drop(columns = 'index', inplace = True)
fresh['prob_w_f'] = prob_w_f

rotten = pd.DataFrame(pd.Series(words)).reset_index()
rotten.drop(columns = 'index', inplace = True)
rotten['prob_w_r'] = prob_w_r

fresh.rename(columns = {0 : 'words', 'prob_w_f' : 'Pr(W|F)'}, inplace = True)
rotten.rename(columns = {0 : 'words', 'prob_w_r' : 'Pr(W|R)'}, inplace = True)

In [307]:
fresh.sort_values(by = 'Pr(W|F)', ascending  = False).head(5)



,words,Pr(W|F)
14621,regenerate,0.0
14615,regains,0.0
6290,facetiousness,0.0
14612,refused,0.0
6294,facilitate,0.0
14607,refried,0.0
6296,facing,0.0
14604,refrained,0.0
14602,reformation,0.0
20491,zzzzzzzzz,0.0


In [306]:
rotten.sort_values(by = 'Pr(W|R)', ascending  = False).head(5)

,words,Pr(W|R)
9450,inventor,0.0
9451,inversion,0.0
9455,investigated,0.0
9457,investigations,0.0
9459,investiture,0.0
9461,investors,0.0
9463,inveterate,0.0
9465,invigorating,0.0
9466,invigoratingly,0.0
10246,lehman,0.0


In [312]:
print(rotten.isna().sum())
print(fresh.isna().sum())

words      0
Pr(W|R)    0
dtype: int64
words      0
Pr(W|F)    0
dtype: int64


In [313]:
#Calculating log probabilities
#Log likelihood of Pr(W|F)
fresh['log_Pr_W_F']=c['Pr(W|F)'].apply(lambda x: np.log(x))


#Log likelihood of Pr(W|R)
rotten['log_Pr_W_R']=c['Pr(W|R)'].apply(lambda x: np.log(x))



In [318]:
#Removing inf and -inf probability values
fresh = fresh.replace(to_replace = [np.inf, -np.inf], value = [np.nan, np.nan]).dropna()
fresh.head(5)

rotten = rotten.replace(to_replace = [np.inf, -np.inf], value = [np.nan, np.nan]).dropna()
rotten.head(5)

,words,Pr(W|F),log_Pr_W_F
0,aaron,0.000473,-7.655548
1,abandon,0.000158,-8.754161
2,abandoned,0.000158,-8.754161
3,abandonment,0.000158,-8.754161
5,abbott,0.000316,-8.061014
6,abbreviated,0.000158,-8.754161
8,abduct,0.000158,-8.754161
9,abe,0.000316,-8.061014
12,abiding,0.000158,-8.754161
13,abilities,0.000316,-8.061014


### Log likelihood

### Fresh Log Likelihood

In [517]:
#Creating a dataframe with vectorized test words
test_words_df = pd.DataFrame(X_test, columns = words)
test_words_df.head()

,aaron,abandon,abandoned,abandonment,abandons,abbott,abbreviated,abdominal,abduct,abe,...,zoom,zooming,zooms,zorro,zorros,zowie,zucker,zweibel,zwick,zzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [520]:
#Transposing fresh
fresh_T =  fresh.T
fresh_T.columns = fresh_T.iloc[0]
fresh_T.drop(['words', 'Pr(W|F)'], inplace = True)

In [521]:
#Displaying fresh
fresh_T

words,aaron,abandon,abandoned,abandonment,abbott,abbreviated,abduct,abe,abiding,abilities,...,zonca,zone,zoological,zooming,zooms,zorro,zorros,zowie,zucker,zwick
log_Pr_W_F,-7.65555,-8.75416,-8.75416,-8.75416,-8.06101,-8.75416,-8.75416,-8.06101,-8.75416,-8.06101,...,-8.75416,-7.65555,-8.75416,-8.75416,-8.75416,-8.06101,-8.75416,-8.75416,-8.06101,-8.06101


In [567]:
#Calculating the log likelihoods for each of the words for fresh
log_word_fresh_list = fresh_T.loc['log_Pr_W_F']
log_word_fresh = pd.DataFrame(columns = test_words_df.columns)
log_word_fresh = log_word_fresh.append(log_word_fresh_list)
log_word_fresh.replace(np.nan, 0, inplace = True)

test_words_df['log_fresh_pred'] = test_words_df.apply(lambda x : x* log_word_fresh.loc['log_Pr_W_F'] ,axis = 1).sum(axis = 1)
test_words_df.head(5)

,aaron,abandon,abandoned,abandonment,abandons,abbott,abbreviated,abdominal,abduct,abe,...,zooming,zooms,zorro,zorros,zowie,zucker,zweibel,zwick,zzzzzzzzz,log_fresh_pred
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-57.607070
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-84.184092
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-61.345003
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-99.249476
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-102.630409


In [570]:
#Transposing rotten
rotten_T =  rotten.T
rotten_T.columns = rotten_T.iloc[0]
rotten_T.drop(['words', 'Pr(W|R)'], inplace = True)
rotten_T

words,abandon,abandons,abdominal,abets,ability,able,abominable,abomination,abortion,abounds,...,zingy,zinnemann,zipper,zippo,zombie,zone,zoom,zweibel,zwick,zzzzzzzzz
log_Pr_W_R,-7.58095,-8.2741,-8.2741,-7.58095,-6.07688,-5.7892,-8.2741,-8.2741,-8.2741,-8.2741,...,-8.2741,-8.2741,-8.2741,-8.2741,-6.32819,-6.07688,-8.2741,-8.2741,-8.2741,-8.2741


In [571]:
#Calculating the log likelihoods for each of the words for rotten
log_word_rotten_list = rotten_T.loc['log_Pr_W_R']
log_word_rotten = pd.DataFrame(columns = test_words_df.columns)
log_word_rotten = log_word_rotten.append(log_word_rotten_list)
log_word_rotten.replace(np.nan, 0, inplace = True)

test_words_df['log_rotten_pred'] = (test_words_df.
                                    apply(lambda x : x* log_word_rotten.loc['log_Pr_W_R'] ,axis = 1).sum(axis = 1))
test_words_df.head()

,aaron,abandon,abandoned,abandonment,abandons,abbott,abbreviated,abdominal,abduct,abe,...,zooms,zorro,zorros,zowie,zucker,zweibel,zwick,zzzzzzzzz,log_fresh_pred,log_rotten_pred
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-57.607070,-57.186279
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-84.184092,-80.254668
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-61.345003,-56.089703
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-99.249476,-104.273027
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-102.630409,-101.680242


In [575]:
#Predicting whether test words belong to fresh or rotten based on log likelihoods
test_words_df['y_pred'] = np.where((test_words_df['log_fresh_pred'] > test_words_df['log_rotten_pred']), 'fresh', 'rotten')
test_words_df.head()

,aaron,abandon,abandoned,abandonment,abandons,abbott,abbreviated,abdominal,abduct,abe,...,zorro,zorros,zowie,zucker,zweibel,zwick,zzzzzzzzz,log_fresh_pred,log_rotten_pred,y_pred
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-57.607070,-57.186279,rotten
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-84.184092,-80.254668,rotten
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-61.345003,-56.089703,rotten
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-99.249476,-104.273027,fresh
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-102.630409,-101.680242,rotten


In [576]:
#Printing the confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

conf = confusion_matrix(y_test['fresh'], test_words_df['y_pred'])
conf


array([[639, 979],
       [339, 608]], dtype=int64)

In [579]:
#Accuracy of fit
from sklearn.metrics import accuracy_score, precision_score, recall_score
accuracy_score(y_test['fresh'], test_words_df['y_pred'])

0.4861598440545809

In [581]:
#Precision
608/(608+979)

0.38311279143037175

In [582]:
#Recall
608/(608+339)

0.6420274551214361

### Frequent Words

In [614]:
test_words = test_words_df.drop(columns = ['log_fresh_pred', 'log_rotten_pred', 'y_pred'])
freq_df = pd.DataFrame(test_words.sum().sort_values(ascending = False), columns = ['freq']).reset_index()
freq_df.shape

(20492, 2)

In [617]:
#Conditional probability of word given fresh for most frequent words

pd.merge(freq_df, fresh, left_on = 'index', right_on = 'words').head(10)

,index,freq,words,Pr(W|F),log_Pr_W_F
0,film,398,film,0.163011,-1.813938
1,movie,354,movie,0.130030,-2.039990
2,one,217,one,0.088685,-2.422659
3,like,149,like,0.043869,-3.126540
4,story,119,story,0.044501,-3.112254
5,much,112,much,0.028089,-3.572377
6,good,104,good,0.046868,-3.060429
7,comedy,97,comedy,0.034559,-3.365089
8,even,94,even,0.034401,-3.369666
9,director,90,director,0.035032,-3.351483


In [618]:
#Conditional probability of word given rotten for most frequent words
pd.merge(freq_df, rotten, left_on = 'index', right_on = 'words').head(10)

,index,freq,words,Pr(W|R),log_Pr_W_R
0,film,398,film,0.118847,-2.129916
1,movie,354,movie,0.145116,-1.930222
2,one,217,one,0.071155,-2.642890
3,like,149,like,0.070645,-2.650084
4,story,119,story,0.042846,-3.150138
5,much,112,much,0.053558,-2.926994
6,good,104,good,0.041061,-3.192698
7,comedy,97,comedy,0.041826,-3.174236
8,even,94,even,0.041571,-3.180352
9,director,90,director,0.034940,-3.354121


In [598]:
#Most frequest fresh words
test_fresh = (test_words_df[test_words_df['y_pred'] == 'fresh'].
              drop(columns = ['log_fresh_pred', 'log_rotten_pred', 'y_pred']))
test_fresh.sum().sort_values(ascending = False).head(10)

film      165
movie     133
one        88
like       52
much       42
comedy     40
best       40
story      40
even       39
good       36
dtype: int64

In [599]:
#Most frequest rotten words
test_rotten = (test_words_df[test_words_df['y_pred'] == 'rotten'].
              drop(columns = ['log_fresh_pred', 'log_rotten_pred', 'y_pred']))
test_rotten.sum().sort_values(ascending = False).head(10)

film        233
movie       221
one         129
like         97
story        79
much         70
good         68
comedy       57
time         56
director     55
dtype: int64

In [652]:
y_test1 = y_test.reset_index()

In [646]:
fresh_index = test_words_df.y_pred[(test_words_df.y_pred == 'fresh')].index

In [649]:
fresh_index.shape

(978,)

In [655]:
y_test1.drop(columns = 'index', inplace = True)

In [662]:
test_words_df.y_pred

0       rotten
1       rotten
2       rotten
3        fresh
4       rotten
5        fresh
6        fresh
7        fresh
8       rotten
9        fresh
10      rotten
11      rotten
12      rotten
13      rotten
14       fresh
15      rotten
16      rotten
17       fresh
18      rotten
19       fresh
20       fresh
21       fresh
22       fresh
23       fresh
24       fresh
25      rotten
26      rotten
27      rotten
28      rotten
29       fresh
         ...  
2535    rotten
2536    rotten
2537    rotten
2538     fresh
2539    rotten
2540    rotten
2541     fresh
2542     fresh
2543    rotten
2544     fresh
2545    rotten
2546     fresh
2547    rotten
2548    rotten
2549     fresh
2550     fresh
2551    rotten
2552    rotten
2553    rotten
2554    rotten
2555    rotten
2556     fresh
2557    rotten
2558     fresh
2559    rotten
2560    rotten
2561    rotten
2562    rotten
2563    rotten
2564    rotten
Name: y_pred, Length: 2565, dtype: object

In [666]:
misclass = test_words_df[y_test1.fresh != test_words_df.y_pred]

In [670]:
#Misclassified words

misclass.head()

,aaron,abandon,abandoned,abandonment,abandons,abbott,abbreviated,abdominal,abduct,abe,...,zorro,zorros,zowie,zucker,zweibel,zwick,zzzzzzzzz,log_fresh_pred,log_rotten_pred,y_pred
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-57.607070,-57.186279,rotten
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-84.184092,-80.254668,rotten
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-61.345003,-56.089703,rotten
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-74.949268,-69.152084,rotten
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-78.233160,-73.761263,rotten


In [673]:
y_test.head(12)

,fresh
7386,fresh
10086,fresh
3037,fresh
3026,fresh
2692,rotten
10207,fresh
11847,fresh
3404,fresh
3299,rotten
3082,fresh


In [696]:
list1 = [7386, 10086, 3037, 2252, 4148]
data.loc[list].quote

7386     Dark, funny, paranoid, arbitrary, humming with...
10086    Miller is outstanding as a father who is utter...
3037     In tandem, the director and screenwriter build...
2252     A splashy, volatile, crowd- pleasing rock-star...
4148     Mr. Black's screenplay is mean-spirited, but i...
Name: quote, dtype: object

In [698]:
#List of misclassified quotes
for i in list:
    print(i, "", clean_quotes[i])

7386  dark funny paranoid arbitrary humming tamped eroticism love things weird good news
10086  miller outstanding father utterly convinced dating first step towards instant complete moral sexual decay
3037  tandem director screenwriter build palpable suspense boy rivet raising issues forgiveness deserves
2252  splashy volatile crowd pleasing rock star melodrama makes sheer emotional wallop sometimes lacks finesse
4148  mr black screenplay mean spirited earns keep sharp sarcastic dialogue ingenious ways setting story


#### On observig the log likelihoods of these misclassified words for fresh and rotten, we observe that they are more or less same and there's not much difference. Since it is observed that the log likelihood of these words is more for rotten , that's why the model has predicted it as rotten

In [700]:
test_words_df.loc[[0,1,2,10,11]][['log_fresh_pred', 'log_rotten_pred']]

 

,log_fresh_pred,log_rotten_pred
0,-57.607070,-57.186279
1,-84.184092,-80.254668
2,-61.345003,-56.089703
10,-74.949268,-69.152084
11,-78.233160,-73.761263


###  NB with smoothing

####  Create two functions: one for fitting NB model, and another to predict outcome based on the tted model. As mentioned above, the model is fully described with 4 probabilities, so your fitting function may return such a list as the model; and the prediction function may take it as an input.

In [ ]:
#Preprocessing and cleaning function
def pre_processing_and_cleaning(file):
    data=pd.read_csv(file)
    data = data[data['fresh'] != 'none']
    data = data.drop_duplicates()
    data = data.drop(columns = 'index')
    return data

In [ ]:
def quote_to_words(raw_quote):
        import re
        from nltk.corpus import stopwords
        #removing raw letters,numbers,punctuations
        letters = re.sub("[^a-zA-Z]"," ",raw_quote)
        #creating an array , resolving whitespaces
        words = letters.lower().split()
        #create an array of stopwords so that we don't have to access corpus to search for a stopword
        stop = set(stopwords.words("english"))
        #removing stopwords from the raw_review
        meaningful_words = [w for w in words if w not in stop]
        #return a string with only the words that are important
        return(" ".join(meaningful_words))
        

In [ ]:
clean_quotes = []

for i in range(data.quote.size):
    clean_quotes.append(quote_to_words(data.quote[i]))

In [848]:
print(index_fresh)

Int64Index([    0,     4,     5,     6,     7,     8,     9,    10,    11,
               12,
            ...
            10242, 10243, 10244, 10245, 10246, 10247, 10250, 10251, 10254,
            10255],
           dtype='int64', length=6337)


In [849]:
#Function to fit NB model which outputs the 4 probabilites : Fresh , rotten , word|fresh, word|rotten
def NB_fit(X_train,y_train, alpha):

    #Probabilities of fresh with alpha values
    prob_fresh = (y_train.fresh[y_train.fresh == 'fresh'].count()+ alpha)/(len(y_train)+alpha)
    #Probabilities of rotten
    prob_rotten = (y_train.fresh[y_train.fresh == 'rotten'].count()+alpha)/(len(y_train)+alpha)
    
    #Log probabilities of fresh
    log_prob_fresh = np.log(prob_fresh)
    
    #log probabilities of rotten
    log_prob_rotten = np.log(prob_rotten)
    
    index_fresh = y_train[y_train['fresh'] == 'fresh'].index
    
    index_rotten = y_train[y_train['fresh'] == 'rotten'].index
    
    X_train_fresh = X_train[index_fresh].sum(axis = 0)
    #Probability of word given fresh
    prob_w_f = (X_train_fresh+alpha)/(len(y_train[y_train['fresh'] == 'fresh'])+alpha)
    
    X_train_rotten = X_train[index_rotten].sum(axis = 0)
    #Probability of word given rotten
    prob_w_r = (X_train_rotten+alpha)/(len(y_train[y_train['fresh'] == 'rotten'])+alpha)
    
    fresh = pd.DataFrame(pd.Series(words)).reset_index()
    fresh.drop(columns = 'index', inplace = True)
    fresh['prob_w_f'] = prob_w_f

    rotten = pd.DataFrame(pd.Series(words)).reset_index()
    rotten.drop(columns = 'index', inplace = True)
    rotten['prob_w_r'] = prob_w_r

    fresh.rename(columns = {0 : 'words', 'prob_w_f' : 'Pr(W|F)'}, inplace = True)
    rotten.rename(columns = {0 : 'words', 'prob_w_r' : 'Pr(W|R)'}, inplace = True)
    
    fresh['log_Pr_W_F']=fresh['Pr(W|F)'].apply(lambda x: np.log(x))
    rotten['log_Pr_W_R']=rotten['Pr(W|R)'].apply(lambda x: np.log(x))
    fresh = fresh.replace(to_replace = [np.inf, -np.inf], value = [np.nan, np.nan]).dropna()
    rotten = rotten.replace(to_replace = [np.inf, -np.inf], value = [np.nan, np.nan]).dropna()
    fresh_T =  fresh.T
    fresh_T.columns = fresh_T.iloc[0]
    fresh_T.drop(['words', 'Pr(W|F)'], inplace = True)
    
    rotten_T =  rotten.T
    rotten_T.columns = rotten_T.iloc[0]
    rotten_T.drop(['words', 'Pr(W|R)'], inplace = True)
    
    log_word_fresh_list = fresh_T.loc['log_Pr_W_F']
    log_word_fresh = pd.DataFrame(columns = test_words_df.columns)
    log_word_fresh = log_word_fresh.append(log_word_fresh_list)
    
    
    log_word_rotten_list = rotten_T.loc['log_Pr_W_R']
    log_word_rotten = pd.DataFrame(columns = test_words_df.columns)
    log_word_rotten = log_word_rotten.append(log_word_rotten_list)
    
    log_word_rotten.replace(np.nan, 0, inplace = True)
    log_word_fresh.replace(np.nan, 0, inplace = True)
    
    
    return log_prob_fresh, log_prob_rotten, log_word_rotten, log_word_fresh

#### 2. Add smoothing to the model. See Schutt p 103 and 109. Smoothing amounts to assuming that we have seen every possible word α > 0 times already, for both classes. (If you wish, you can also assume you have seen the words α times for F and β times for R). Note that α does not have to be an integer, and typically the best α < 1.


In [756]:
# Adding smoothing by passing alpha=0
log_prob_fresh, log_prob_rotten, log_word_rotten, log_word_fresh=NB_fit(X_train,y_train, 0.1)

In [ ]:
#Function to calculate accuracy of the fit

def predict(X_test, y_test, log_word_fresh, log_word_rotten):
    from sklearn.metrics import accuracy_score
    test_words_df = pd.DataFrame(X_test, columns = words)
    test_words_df['log_fresh_pred'] = test_words_df.apply(lambda x : x* log_word_fresh.loc['log_Pr_W_F'] ,axis = 1).sum(axis = 1)
    test_words_df['log_rotten_pred'] = (test_words_df.apply(lambda x : x* log_word_rotten.loc['log_Pr_W_R'] ,axis = 1).sum(axis = 1))
    test_words_df['y_pred'] = np.where((test_words_df['log_fresh_pred'] > test_words_df['log_rotten_pred']), 'fresh', 'rotten')
    
    return accuracy_score(y_test['fresh'], test_words_df['y_pred'])
    

In [765]:
#Accuracy
predict(X_test, y_test, log_word_fresh, log_word_rotten)

0.7461988304093568

## Cross Validation

In [863]:
def kfold_cross_val(k, X, y, alpha):
     # Shuffle Indices
    #np.random.seed(42)
    #shuffled_indices = np.random.permutation(len(X))
    #X = X[shuffled_indices]
    #y = y[shuffled_indices]
    
    # Splitting predictors and predictands into k folds
    X_split = np.array_split(X, k)
    y_split = np.array_split(y, k)
    
    # Intializing the classification scores
    accuracy_sum = 0
    
    for i in range(k):
        X_train = X_split.copy() # Creating a copy of the predictors
        X_valid = X_split[i]     # Validation predictors
        y_train = y_split.copy() # Copy of the target
        y_valid = y_split[i]     # Validation target
        del X_train[i]           # Deleting the validation predictors from the training
        del y_train[i]           # Deleting the validation target from the training
        
        # Concatenate the folds into single array
        X_train = np.concatenate(X_train)
        y_train = np.concatenate(y_train)
        
        # Performing the model and getting classification scores after each iteration
        acc = perform(i, X_train, pd.DataFrame(y_train, columns = ['fresh']).reset_index().drop(columns = ['index']), X_valid, pd.DataFrame(y_valid, columns = ['fresh']).reset_index().drop(columns = ['index']), alpha)
        
        # Summing the classification scores after each iteration
        accuracy_sum = accuracy_sum + acc
    
    print("Average accuracy:", round((accuracy_sum/k), 2))

In [864]:
def perform(a, X_train, y_train, X_valid, y_valid, alpha):
    from sklearn import metrics # importing the metrics library to calculate classification scores
    # Fitting the model with the training data
    log_pr_f, log_pr_r, log_pr_w_r, log_pr_w_f = NB_fit(X_train, y_train, alpha)
    
    # Predicting and Calculating the accuracy
    accuracy = predict(X_valid, y_valid, log_pr_w_f, log_pr_w_r)
    
    print(str(a + 1), "Accuracy:", round(accuracy, 2))
    return accuracy

In [865]:
kfold_cross_val(5, X_train, np.array(y_train), 0.1)

1 Accuracy: 0.73
2 Accuracy: 0.72
3 Accuracy: 0.74
4 Accuracy: 0.71
5 Accuracy: 0.74
Average accuracy: 0.73


In [856]:
#Using CV with inbuilt lib
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

nb_clf = MultinomialNB(alpha = 0.3)
nb_clf.fit(X_train, y_train)

cross_val_score(nb_clf, X_test, y_test, cv = 3)

C:\Users\roshn\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\roshn\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\roshn\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\roshn\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

array([0.7067757 , 0.71929825, 0.70140515])

In [ ]:
#Calculating the accuracy with different alpha val

alpha = [0.1, 0.3, 0.5]

for i in alpha:
    print("alpha", i)
    kfold_cross_val(3, X_train, np.array(y_train), i)